<a href="https://colab.research.google.com/github/Talha1818/Documents-Classification-LayoutLMV3/blob/master/Model_Inference_LayoutLMV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model Inference

In [ ]:
!pip install PyPDF4~=1.27.0
!pip install pdf2image~=1.16.0
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install poppler-utils

!pip install -qqq transformers --progress-bar off
!pip install -qqq easyocr --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61226 sha256=2319118566b2b6303bd2694b21e249378d04fa4ac036465cb2a9a0febce4c22d
  Stored in directory: /root/.cache/pip/wheels/83/cc/14/cb307e5c99235c4497c7895cdb60b4f7ba2a738b6a5fc0d423
Successfully built PyPDF4
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubu

In [ ]:
import csv
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from PyPDF4.pdf import BytesIO
import os
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output
import io

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
pd.set_option('display.max_columns', None)

from tqdm.auto import tqdm
import os

import warnings
warnings.filterwarnings('ignore')

from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
import torch
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from sklearn.model_selection import train_test_split
# import imgkit
import easyocr
import torchvision.transforms as T
from pathlib import Path
import matplotlib.pyplot as plt
import os
import cv2
from typing import List
import json

from pathlib import Path


In [ ]:
from pdf2image import convert_from_path
from PIL import Image

def delete_png_and_json_files(directory):
    # Iterate over files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a PNG or JSON file
        if filename.endswith(".jpg") or filename.endswith(".json"):
            # Construct the full file path
            file_path = os.path.join(directory, filename)
            try:
                # Delete the file
                os.remove(file_path)
                # print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Failed to delete {file_path}: {e}")

def get_images_from_file(file_path, path='', filename='', save=False):
    try:
        delete_png_and_json_files(path)

        # Extract images from PDF
        images = convert_from_path(file_path)
        # print(f"{path}/{filename}: Total Images:: {len(images)}")
        if save:
          # Save each image
          for i, image in enumerate(images):
              image.save(f"{path}/{filename}__page{i+1}.jpg", "JPEG")

        # return images, len(images)
        # return len(images)
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
def create_bounding_box(bbox_data):
    xs = []
    ys = []
    for x, y in bbox_data:
        xs.append(x)
        ys.append(y)

    left = int(min(xs))
    top = int(min(ys))
    right = int(max(xs))
    bottom = int(max(ys))

    return [left, top, right, bottom]

def scale_bounding_box(box: List[int], width_scale : float = 1.0, height_scale : float = 1.0) -> List[int]:
    return [
        int(box[0] * width_scale),
        int(box[1] * height_scale),
        int(box[2] * width_scale),
        int(box[3] * height_scale)
    ]

In [ ]:
def make_json_data(image_saving_path, image_paths_list):
  reader = easyocr.Reader(['en'])
  image_paths = image_paths_list

  for image_path in tqdm(image_paths):
    ocr_result = reader.readtext(str(image_path))

    ocr_page = []
    for bbox, word, confidence in ocr_result:
        ocr_page.append({
            "word": word, "bounding_box": create_bounding_box(bbox)
        })

    # Convert image_path to a Path object
    # image_path = Path(image_path)
    # print(image_path)

    with image_path.with_suffix(".json").open("w") as f:
        json.dump(ocr_page, f)

In [ ]:
import torch.nn.functional as F

def predict_document_image(
    image_path: Path,
    model: LayoutLMv3ForSequenceClassification,
    processor: LayoutLMv3Processor):

    json_path = image_path.with_suffix(".json")
    with json_path.open("r") as f:
        ocr_result = json.load(f)

        with Image.open(image_path).convert("RGB") as image:

            width, height = image.size
            width_scale = 1000 / width
            height_scale = 1000 / height

            words = []
            boxes = []
            for row in ocr_result:
                boxes.append(
                    scale_bounding_box(
                        row["bounding_box"],
                        width_scale,
                        height_scale
                    )
                )
                words.append(row["word"])

            encoding = processor(
                image,
                words,
                boxes=boxes,
                max_length=512,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

    with torch.inference_mode():
        output = model(
            input_ids=encoding["input_ids"].to(DEVICE),
            attention_mask=encoding["attention_mask"].to(DEVICE),
            bbox=encoding["bbox"].to(DEVICE),
            pixel_values=encoding["pixel_values"].to(DEVICE)
        )

    # Apply softmax to obtain probability scores
    probabilities = F.softmax(output.logits, dim=1)

    predicted_class = output.logits.argmax()
    predicted_label = model.config.id2label[predicted_class.item()]

    return predicted_label, probabilities.tolist()[0]


In [ ]:
def get_results(test_images):
  from tqdm.auto import tqdm
  from statistics import mode

  predictions = []
  probabilities_list = []
  for image_path in test_images:
      predicted_label, probabilities = predict_document_image(
          image_path,
          model,
          processor
      )
      predictions.append(predicted_label)
      probabilities_list.append(probabilities)

  # Calculate mode of predictions
  final_prediction = mode(predictions)

  DIC  = {
    "pages": [f'page_{i+1}' for i in range(len(predictions))],
    "prediction": predictions,
    "probability": [round(max(i),4) for i in probabilities_list],
     'final-prediction': [final_prediction] + [''] * (len(predictions) - 1),
  }
  testing_df = pd.DataFrame(DIC)
  avg_prob = testing_df[testing_df['prediction']==final_prediction][['probability']].mean().values[0]
  testing_df['Avg-Probability'] = [round(avg_prob, 4)] + [''] * (len(predictions) - 1)
  return testing_df

# Prediction

In [ ]:
import time

start = time.time()
pt_save_directory = "/content/drive/MyDrive/DMS_Model_Files/Model"

feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
tokenizer = LayoutLMv3TokenizerFast.from_pretrained(pt_save_directory, ignore_mismatched_sizes=True)
processor = LayoutLMv3Processor(feature_extractor, tokenizer)
loaded_model = LayoutLMv3ForSequenceClassification.from_pretrained(pt_save_directory)
print("Model Loaded.")

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)
model = loaded_model.eval().to(DEVICE)

end = time.time()
print(f"Model Loaded in {end - start} seconds")

Model Loaded.
cuda:0
Model Loaded in 22.102764129638672 seconds


In [ ]:
start = time.time()
pdf_file = "/content/drive/MyDrive/Folder Sorts/Bills/Gas/20160516-AGL-57611618ed5005c2067f7aa0.pdf"
image_saving_path = '/content'
get_images_from_file(pdf_file, path=image_saving_path, filename='sample', save=True)

image_paths = list(Path(image_saving_path).glob("*.jpg"))
make_json_data(image_saving_path, image_paths)
print(get_results(image_paths))

end = time.time()
print(f"\nGet Results in {end - start} seconds")

  0%|          | 0/2 [00:00<?, ?it/s]

    pages prediction  probability final-prediction Avg-Probability
0  page_1   Gas Bill       0.9918         Gas Bill           0.992
1  page_2   Gas Bill       0.9922                                 

Get Results in 12.443883657455444 seconds


In [ ]:
get_results(image_paths)

,pages,prediction,probability,final-prediction,Avg-Probability
0,page_1,Gas Bill,0.9918,Gas Bill,0.992
1,page_2,Gas Bill,0.9922,,


# Testing on Unseen Data

In [ ]:
import os

# Define the folder path
folder_path = "/content/drive/MyDrive/samle-invoice"

# Get all files in the folder
files = os.listdir(folder_path)

# Filter only the PDF files
pdf_files = [file for file in files if file.endswith(".pdf")]

dataframe = []
# Print the list of PDF files
print("PDF files in the folder:")
for pdf_file in pdf_files:
    # pdf_file = "/content/drive/MyDrive/Folder Sorts/Bills/Gas/20160516-AGL-57611618ed5005c2067f7aa0.pdf"
    image_saving_path = '/content'
    get_images_from_file("/content/drive/MyDrive/samle-invoice/"+pdf_file, path=image_saving_path, filename='sample', save=True)

    image_paths = list(Path(image_saving_path).glob("*.jpg"))
    make_json_data(image_saving_path, image_paths)
    print()
    print("File Name:", pdf_file)
    res = get_results(image_paths)
    print(res)
    res['Pdf_file'] = pdf_file
    dataframe.append(res)
    print()

PDF files in the folder:


  0%|          | 0/2 [00:00<?, ?it/s]


File Name: 20010101-Ausmedic-534b42c2e4b003df37897b1f.pdf
    pages prediction  probability final-prediction Avg-Probability
0  page_1    Invoice       0.9986          Invoice          0.9981
1  page_2    Invoice       0.9976                                 



  0%|          | 0/11 [00:00<?, ?it/s]


File Name: 19900510-Adroit Insurance Amjik MV-52910f4de4b05d2fe805c5a6.pdf
      pages      prediction  probability final-prediction Avg-Probability
0    page_1  Insurance Bill       0.9984   Insurance Bill          0.9984
1    page_2  Insurance Bill       0.9984                                 
2    page_3  Insurance Bill       0.9983                                 
3    page_4  Insurance Bill       0.9984                                 
4    page_5  Insurance Bill       0.9984                                 
5    page_6  Insurance Bill       0.9984                                 
6    page_7  Insurance Bill       0.9984                                 
7    page_8  Insurance Bill       0.9983                                 
8    page_9  Insurance Bill       0.9985                                 
9   page_10  Insurance Bill       0.9984                                 
10  page_11  Insurance Bill       0.9984                                 



...


File Name: 20010101-Costco Wholesale-62b50dcbe4b05136b4468df7.pdf
    pages prediction  probability final-prediction  Avg-Probability
0  page_1    Receipt       0.9983          Receipt           0.9983



  0%|          | 0/1 [00:00<?, ?it/s]


File Name: 20010101-AGL-633124f9e4b0e87ba4ec523a.pdf
    pages prediction  probability final-prediction  Avg-Probability
0  page_1   Gas Bill       0.9903         Gas Bill           0.9903



  0%|          | 0/1 [00:00<?, ?it/s]


File Name: 20010101-Chemist Warehouse-64aca3c3e4b0f869a8be6f5c.pdf
    pages prediction  probability final-prediction  Avg-Probability
0  page_1    Invoice       0.9045          Invoice           0.9045



In [ ]:
final_res = pd.concat(dataframe)
final_res.head()

,pages,prediction,probability,final-prediction,Avg-Probability,Pdf_file
0,page_1,Invoice,0.9986,Invoice,0.9981,20010101-Ausmedic-534b42c2e4b003df37897b1f.pdf
1,page_2,Invoice,0.9976,,,20010101-Ausmedic-534b42c2e4b003df37897b1f.pdf
0,page_1,Insurance Bill,0.9984,Insurance Bill,0.9984,19900510-Adroit Insurance Amjik MV-52910f4de4b...
1,page_2,Insurance Bill,0.9984,,,19900510-Adroit Insurance Amjik MV-52910f4de4b...
2,page_3,Insurance Bill,0.9983,,,19900510-Adroit Insurance Amjik MV-52910f4de4b...


In [ ]:
final_res.to_csv("/content/drive/MyDrive/Documents_Files_DMS/unseen_data.csv", index=False)